1° PROYECTO INDIVIDUAL MLOps -1er parte
ETL

1) Importación de librerías

In [482]:
import pandas as pd
import numpy as np
import locale
import re
locale.setlocale(locale.LC_TIME, 'es_ES')

'es_ES'

2) Lectura de archivo csv como dataframe

In [483]:
df= pd.read_csv(r"C:/Users/Economia/Documents/Soy Henry/Proyecto Individual/movies_dataset.csv", sep=",", encoding = "utf-8")

C:\Users\Economia\AppData\Local\Temp\ipykernel_18720\3752448005.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv(r"C:/Users/Economia/Documents/Soy Henry/Proyecto Individual/movies_dataset.csv", sep=",", encoding = "utf-8")


3) Transformaciones


a) Expansión de columnas anidadas

In [484]:
invalid_ids = []  # Lista para almacenar los valores no válidos

# Expresión regular para verificar si un string contiene solo dígitos
regex = r'^\d+$'

# Iterar sobre los valores de la columna 'id' y verificar si cumplen con el patrón
for value in df['id']:
    if not re.match(regex, str(value)):
        # El valor no cumple con el patrón, agregarlo a la lista de valores no válidos
        invalid_ids.append(value)

# Imprimir la lista de valores no válidos
print("Valores no válidos en la columna 'id':", invalid_ids)

Valores no válidos en la columna 'id': ['1997-08-20', '2012-09-29', '2014-01-01']


In [485]:
# Índices de las filas que deseas eliminar
indices_eliminar = [19730, 29503, 35587]

# Eliminar las filas utilizando el método 'drop'
df = df.drop(indices_eliminar)

In [486]:
df3 = pd.DataFrame(df[['genres',"id"]])

In [487]:
df3['genres'] = df3['genres'].replace({r'\[|\]': ''}, regex=True)

In [488]:
# Utilizar la función findall para separar los elementos en filas individuales
df3['genres'] = df3['genres'].str.findall(r"\{.*?\}")

# Explotar los elementos de la columna 'genres' en registros individuales
df4 = df3.explode('genres')

In [489]:
df4['genres'] = df4['genres'].replace({r'{|}': ''}, regex=True).to_frame()

In [490]:
df4[['id_genres', 'name_genres']] = df4['genres'].str.extract(r"'id': (\d+), 'name': '([^']+)'")

In [491]:
df4.drop('genres', axis=1, inplace=True)

In [492]:
df4['name_genres'] = df4['name_genres'].astype(str)
df4_1 = df4.groupby(df4.index).agg({'name_genres': ', '.join, 'id': 'first'})

In [493]:
df4_1 

,name_genres,id
0,"Animation, Comedy, Family",862
1,"Adventure, Fantasy, Family",8844
2,"Romance, Comedy",15602
3,"Comedy, Drama, Romance",31357
4,Comedy,11862
...,...,...
45461,"Drama, Family",439050
45462,Drama,111109
45463,"Action, Drama, Thriller",67758
45464,nan,227506


In [494]:
df['revenue'].fillna(0, inplace=True)

In [495]:
df['budget'].fillna(0, inplace=True)

b) Modificaciones en datos de tipo fecha

In [496]:
df['release_date'].dropna(inplace=True)

In [497]:
df['release_date_copy'] = df['release_date']

In [498]:
# Función para corregir el formato de las fechas
def corregir_formato_fecha(date):
    if date == 'Sin Fecha':
        return date
    try:
        return pd.to_datetime(date, format='%Y-%m-%d')
    except ValueError:
        return 'Formato Inválido'

In [499]:
# Aplicar la función para corregir el formato de las fechas en la columna 'release_date_copy'
df['release_date_copy'] = df['release_date_copy'].apply(corregir_formato_fecha)

# Ver los registros que no cumplen el formato válido
registros_formato_invalido = df[df['release_date_copy'] == 'Formato Inválido']

In [500]:
df['release_year'] = df['release_date_copy'].apply(lambda x: x.year if isinstance(x, pd.Timestamp) else None).astype('Int64')

In [501]:
df['release_month'] = df['release_date_copy'].apply(lambda x: x.month if isinstance(x, pd.Timestamp) else None).astype('Int64')

In [502]:
df['release_day'] = df['release_date_copy'].apply(lambda x: x.day if isinstance(x, pd.Timestamp) else None).astype('Int64')

In [503]:
df = df.drop('release_date', axis=1) #elimino la columna release_date original que tiene fechas en formatos incorrectos

In [504]:
df.rename(columns={'release_date_copy': 'release_date'}, inplace=True)

In [505]:
# Función para corregir el formato de la fecha
def corregir_formato_fecha(date):
    if isinstance(date, pd.Timestamp):
        return date.strftime('%Y-%m-%d')
    else:
        return date

# Aplicar la función para corregir el formato de la columna 'release_date'
df['release_date'] = df['release_date'].apply(corregir_formato_fecha)

In [506]:
meses_dict = {
    1: 'enero',
    2: 'febrero',
    3: 'marzo',
    4: 'abril',
    5: 'mayo',
    6: 'junio',
    7: 'julio',
    8: 'agosto',
    9: 'septiembre',
    10: 'octubre',
    11: 'noviembre',
    12: 'diciembre'
}

df['release_month'] = df['release_month'].map(meses_dict)

In [507]:
# Inicializar la columna 'day_of_week'
df['day_of_week'] = ''

# Recorrer los valores de la columna 'release_date' y obtener el día de la semana en español
for i, date in enumerate(df['release_date']):
    try:
        dt = pd.to_datetime(date)
        day_of_week_spanish = dt.strftime('%A').capitalize()
        df.at[i, 'day_of_week'] = day_of_week_spanish
    except ValueError:
        df.at[i, 'day_of_week'] = 'Formato inválido'

Convierto y controlo que las variables numericas

In [508]:
df['budget'] = pd.to_numeric(df['budget'], errors='coerce')

In [509]:
# Crear la columna "return" con la división entre "revenue" y "budget"
df['return'] = df['revenue'] / df['budget']

# Reemplazar los valores NaN en la columna "return" con 0
df['return'].fillna(0, inplace=True)

In [510]:
# Reemplazar los valores "inf" en la columna "return" por 0
df['return'].replace([np.inf, -np.inf], 0, inplace=True)

Eliminación de columnas innecesarias

In [511]:
columns_1= ['video',"imdb_id", "adult", "original_title","poster_path", "homepage"]
df = df.drop(columns_1, axis=1) #elimino la columna innecesarias

Armo un dataset con las columnas ya editadas 

In [512]:
# Crear el nuevo dataframe con las columnas deseadas
df_final = df[['id', 'day_of_week', 'return', 'release_month', 'release_year', 'title', 'popularity', 'vote_count', 'vote_average', 'release_date', 'budget', 'revenue']].copy()

In [513]:
df_final.columns


Index(['id', 'day_of_week', 'return', 'release_month', 'release_year', 'title',
       'popularity', 'vote_count', 'vote_average', 'release_date', 'budget',
       'revenue'],
      dtype='object')

Ahora trabajamos con la columna cast del dataset credit

Expansión de columnas anidadas - proceso de armado de dataset para conocer mas sobre los actores

In [514]:
df_credit = pd.read_csv(r"C:/Users/Economia/Documents/Soy Henry/Proyecto Individual/credits.csv")

In [515]:
df_cast1= pd.DataFrame(df_credit[['cast', 'id']])

In [516]:
df_cast1['cast'] = df_cast1['cast'].replace({r'\[|\]': ''}, regex=True)

In [517]:
# Utilizamos la función findall para separar los elementos en filas individuales
df_cast1['cast'] = df_cast1['cast'].str.findall(r"\{.*?\}")

# Separamos (hacemos saltos cuando obsermos las llaves) los elementos de la columna 'genres' en registros individuales
df_cast1 = df_cast1.explode('cast')

In [518]:
df_cast1['cast'] = df_cast1['cast'].replace({r'{|}': ''}, regex=True).to_frame() #eliminamos las llaves

In [519]:
df_cast1_copy= df_cast1

In [520]:
df_desanidado = df_cast1_copy['cast'].str.split(',', expand=True)

In [521]:
df_desanidado['id'] = df_cast1_copy['id']

In [522]:
columnas_eliminar = df_desanidado.columns[8:14]  # Obtiene los nombres de las columnas a eliminar

# Eliminar las columnas especificadas
df_desanidado = df_desanidado.drop(columnas_eliminar, axis=1)

In [523]:
df_desanidado.columns = ['cast_id', 'character', 'credit_id','gender','id_cast', 'name_cast', 'order', 'profile_path' , "id"]

In [524]:
df_desanidado['cast_id'] = df_desanidado['cast_id'].str.replace("'cast_id':", '')
df_desanidado['character'] = df_desanidado['character'].str.replace("'character':", '')
df_desanidado['credit_id'] = df_desanidado['credit_id'].str.replace("'credit_id':", '')
df_desanidado['gender'] = df_desanidado['gender'].str.replace("'gender':", '')
df_desanidado['id_cast'] = df_desanidado['id_cast'].str.replace("'id':", '')
df_desanidado['name_cast'] = df_desanidado['name_cast'].str.replace("'name':", '')
df_desanidado['order'] = df_desanidado['order'].str.replace("'order':", '')
df_desanidado['profile_path'] = df_desanidado['profile_path'].str.replace("'profile_path':", '')

In [ ]:
df_clean = df_desanidado.dropna(subset=df_desanidado.columns.difference(['id']), how='all')

In [ ]:
df_clean.columns

Index(['cast_id', 'character', 'credit_id', 'gender', 'id_cast', 'name_cast',
       'order', 'profile_path', 'id'],
      dtype='object')

Ahora trabajamos con la columna crew del dataset credit


In [ ]:
df_crew= pd.DataFrame(df_credit[['crew', 'id']])

Expansión de columnas anidadas - proceso de armado de dataset para conocer mas sobre los directores

In [ ]:
df_crew['crew'] = df_crew['crew'].replace({r'\[|\]': ''}, regex=True)

In [ ]:
# Utilizar la función findall para separar los elementos en filas individuales
df_crew['crew'] = df_crew['crew'].str.findall(r"\{.*?\}")

# Explotar los elementos de la columna 'genres' en registros individuales
df_crew = df_crew.explode('crew')

In [ ]:
df_crew_copy= df_crew

In [ ]:
df_crew_copy = df_crew_copy['crew'].str.split(',', expand=True)

In [ ]:
df_crew_copy['id'] = df_crew['id']

In [ ]:
df_crew_copy.columns = ['credit_id', 'department', 'gender','id_crew', "job", 'name_crew', 'profile_path' , "raro", "id"]

In [ ]:
#vamos eliminando donde encuentre estos nombres
df_crew_copy['credit_id'] = df_crew_copy['credit_id'].str.replace("'credit_id':", '')
df_crew_copy['department'] = df_crew_copy['department'].str.replace("'department':", '')
df_crew_copy['gender'] = df_crew_copy['gender'].str.replace("'gender':", '')
df_crew_copy['id_crew'] = df_crew_copy['id_crew'].str.replace("'id_crew':", '')
df_crew_copy['job'] = df_crew_copy['job'].str.replace("'job':", '')
df_crew_copy['name_crew'] = df_crew_copy['name_crew'].str.replace("'name':", '')
df_crew_copy['profile_path'] = df_crew_copy['profile_path'].str.replace("'profile_path':", '')

In [ ]:
# Filtrar los registros no nulos en una columna
columna_no_nulos = df_crew_copy['raro'].notna()  # o df['nombre_columna'].notnull()

# Crear un nuevo DataFrame con los registros no nulos
df_no_nulos = df_crew_copy[columna_no_nulos].copy()


In [ ]:
df_no_nulos['name_crew'] = df_no_nulos['name_crew'] + df_no_nulos['profile_path']

In [ ]:
df_no_nulos= df_no_nulos.drop('profile_path', axis=1)

In [ ]:
df_no_nulos = df_no_nulos.rename(columns={'raro': 'profile_path'})

In [ ]:
df_no_nulos['id_crew'] = df_no_nulos['id_crew'].str.replace("'id':", '')
df_no_nulos['profile_path'] = df_no_nulos['profile_path'].str.replace("'profile_path':", '')

In [ ]:
df_crew_copy= df_crew_copy.drop('raro', axis=1)

In [ ]:
df_crew_copy = pd.concat([df_crew_copy, df_no_nulos])

In [ ]:
df_crew_copy.columns

Index(['credit_id', 'department', 'gender', 'id_crew', 'job', 'name_crew',
       'profile_path', 'id'],
      dtype='object')

Elimino todos los registros del dataset de name_crew de personas que no trabajen en el pefil de directores en todas sus diferentes vertientes. Dado que me permite reducir el tamaño de los registros 

In [ ]:
# Filtrar el DataFrame df_crew_copy por el valor "Director" en la columna "job"
df_directors = df_crew_copy[df_crew_copy['job'].str.contains('Director', case=False, na=False)]

# Crear un nuevo DataFrame con las columnas "id" y "name_crew" para los directores
df_director_names = df_directors[['id', 'name_crew']].copy()

In [ ]:
df_director_names['id'] = df_director_names['id'].astype(str)

In [ ]:
df_final = pd.merge(df_final, df_director_names[['id', 'name_crew']], on='id', how='left')

In [ ]:
# Convertir la columna "id" del dataframe df_clean a tipo object
df_clean['id'] = df_clean['id'].astype(str)

# Realizar el merge entre df_final y df_clean utilizando la columna "id"
df_final = pd.merge(df_final, df_clean[['id', 'name_cast']], on='id', how='left')

C:\Users\Economia\AppData\Local\Temp\ipykernel_18720\1830367170.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['id'] = df_clean['id'].astype(str)


In [ ]:
df4['id'] = df4['id'].astype(str)

In [ ]:
df_final = pd.merge(df_final, df4[['id', 'name_genres']], on='id', how='left')

In [ ]:
merged_df = df_final.copy()

In [ ]:
merged_df= merged_df.drop(['name_cast','name_crew','name_genres'], axis=1)

In [ ]:
merged_df = merged_df.merge(df4_1[['id', 'name_genres']], on='id', how='left')


In [ ]:
merged_df.drop_duplicates(inplace=True)  # dataset para hacer el analisis de los datos

Armamos un dataframe con las columnas relevantes que nos permitan tener un registro por pelicula al compartir informacion común. De esta forma reduzco el tamaño de los registros 

In [ ]:
columns = ['id', 'title', 'vote_count', 'vote_average', 'popularity', 'day_of_week', 'release_month', 'release_year', 'return', 'budget','release_date' , 'revenue']
new_df = df_final[columns].copy()

In [ ]:
# Eliminar los datos duplicados
new_df.drop_duplicates(inplace=True)

In [ ]:
new_df.head(2) # datos listos para usar en las primeras 5 funciones 

,id,title,vote_count,vote_average,popularity,day_of_week,release_month,release_year,return,budget,release_date,revenue
0,862,Toy Story,5415.0,7.7,21.946943,Lunes,octubre,1995,12.451801,30000000.0,1995-10-30,373554033.0
156,8844,Jumanji,2413.0,6.9,17.015539,Viernes,diciembre,1995,4.043035,65000000.0,1995-12-15,262797249.0


Armamos un dataset que nos permite estudiar a los actores

In [ ]:
new_df2 = df_final[['name_cast', 'title', 'return','id']].copy()

In [ ]:
# Eliminar los datos duplicados
new_df2.drop_duplicates(inplace=True)

In [ ]:
new_df2.head(5) # datos listos para usar en la funcion get actor

,name_cast,title,return,id
0,'Tom Hanks',Toy Story,12.451801,862
3,'Tim Allen',Toy Story,12.451801,862
6,'Don Rickles',Toy Story,12.451801,862
9,'Jim Varney',Toy Story,12.451801,862
12,'Wallace Shawn',Toy Story,12.451801,862


Armamos un dataset que nos permite estudiar a los directores

In [ ]:
new_df3 = df_final[['name_crew', 'title', 'return', 'release_date', 'budget','revenue', 'id' ]].copy()

In [ ]:
# Eliminar los datos duplicados
new_df3.drop_duplicates(inplace=True)

In [ ]:
new_df3.head(5)  #datos listos para usar en la funcion get_director

,name_crew,title,return,release_date,budget,revenue,id
0,'John Lasseter',Toy Story,12.451801,1995-10-30,30000000.0,373554033.0,862
39,'Ash Brannon',Toy Story,12.451801,1995-10-30,30000000.0,373554033.0,862
78,'William Reeves',Toy Story,12.451801,1995-10-30,30000000.0,373554033.0,862
117,'Rich Quade',Toy Story,12.451801,1995-10-30,30000000.0,373554033.0,862
156,'Joe Johnston',Jumanji,4.043035,1995-12-15,65000000.0,262797249.0,8844


Endpoint

In [ ]:
def cantidad_filmaciones_mes(df, mes):
    # Filtrar las filas que corresponden al mes especificado (aceptar el mes sin comillas)
    peliculas_mes = df[df['release_month'].str.lower() == mes.lower()]

    # Crear una lista de IDs únicos de películas
    ids_unicos = peliculas_mes['id'].unique()

    # Contar la cantidad de películas distintas
    cantidad_peliculas = len(ids_unicos)

    return {'mes':mes, 'cantidad':cantidad_peliculas}

In [ ]:
cantidad_filmaciones_mes(new_df, "octubre")

{'mes': 'octubre', 'cantidad': 4613}

In [ ]:
def score_titulo(df, titulo):
    # Filtrar el DataFrame para obtener la fila correspondiente al título de la película
    pelicula = df[df['title'] == titulo]

    if pelicula.empty:
        return "No se encontró información para la película:", titulo, "Por favor controla que estas escribiendo bien"

    # Obtener el año de estreno y la popularidad de la película
    release_year = pelicula['release_year'].values[0]
    popularity = pelicula['popularity'].values[0]

    return {'titulo':titulo, 'año': round(release_year), 'popularidad':popularity}

In [ ]:
score_titulo (new_df, "Toy Story")

{'titulo': 'Toy Story', 'año': 1995, 'popularidad': 21.946943}

In [ ]:
def votos_titulo(df, titulo:str):
    # Filtrar el DataFrame para obtener la fila correspondiente al título de la película
    pelicula = df[df['title'] == titulo]

    if pelicula.empty:
        return "No se encontró información para la película:", titulo

    # Obtener el año de estreno y la popularidad de la película
    release_year = pelicula['release_year'].values[0]
    vote_count = pelicula['vote_count'].values[0]
    vote_average = pelicula['vote_average'].values[0]

    if vote_count >= 2000:
        return {'titulo':titulo, 'año':round(release_year), 'voto_total':vote_count, 'voto_promedio':vote_average}
    
    else:
        return f"La película {titulo} no cumple con la cantidad mínima de valoraciones requerida."

In [ ]:
votos_titulo(new_df, "Toy Story")

{'titulo': 'Toy Story',
 'año': 1995,
 'voto_total': 5415.0,
 'voto_promedio': 7.7}

In [ ]:
def get_actor(df, nombre):
    # Reemplazar los valores nulos en la columna 'name_cast' con una cadena única
    df['name_cast'] = df['name_cast'].fillna('No disponible')

    # Filtrar las filas que contienen el nombre en la columna 'name_cast'
    peliculas_nombre = df[df['name_cast'].str.contains(nombre, case=False, regex=False)]

    # Obtener la cantidad de películas
    cantidad_peliculas = peliculas_nombre['id'].nunique()

    # Obtener la suma de los retornos
    suma_retornos = peliculas_nombre['return'].sum()

    # Obtener el promedio de los retornos
    promedio_retornos = peliculas_nombre['return'].mean()

    return {'actor':nombre, 'cantidad_filmaciones':cantidad_peliculas,'retorno_total' : suma_retornos, 'retorno_promedio':promedio_retornos}

In [ ]:
get_actor(new_df2,'Tom Hanks')

{'actor': 'Tom Hanks',
 'cantidad_filmaciones': 70,
 'retorno_total': 173.3495309552024,
 'retorno_promedio': 2.476421870788606}

In [ ]:
def get_director(df, nombre_director):
    # Reemplazar los valores nulos en la columna 'name_crew' con una cadena única
    df['name_crew'] = df['name_crew'].fillna('No disponible')

    # Filtrar las filas que contienen el nombre en la columna 'name_crew'
    peliculas_nombre = df[df['name_crew'].str.contains(nombre_director, case=False, regex=False)]

    # Obtener la cantidad de películas
    cantidad_peliculas = peliculas_nombre['id'].nunique()

    # Obtener la suma de los retornos
    suma_retornos = peliculas_nombre.groupby('id')['return'].sum().sum()

    # Crear un DataFrame con el título, la fecha de lanzamiento, el retorno, el costo y la ganancia de las películas
    df_peliculas = peliculas_nombre.drop_duplicates(subset='id')[['title', 'release_date', 'return', 'budget', 'revenue']].reset_index(drop=True)

    return df_peliculas, f"'Nombre del director': {nombre_director}, 'retorno_total_director': {suma_retornos}"

In [ ]:
get_director(new_df3,'John Lasseter')

(                      title release_date     return       budget      revenue
 0                 Toy Story   1995-10-30  12.451801   30000000.0  373554033.0
 1              A Bug's Life   1998-11-25   3.027157  120000000.0  363258859.0
 2               Toy Story 2   1999-10-30   5.526299   90000000.0  497366869.0
 3                  Luxo Jr.   1986-08-17   0.000000          0.0          0.0
 4                      Cars   2006-06-08   3.849860  120000000.0  461983149.0
 5                    Cars 2   2011-06-11   2.799262  200000000.0  559852396.0
 6                   Tin Toy   1988-08-01   0.000000          0.0          0.0
 7               Red's Dream   1987-08-17   0.000000          0.0          0.0
 8               Knick Knack   1989-01-01   0.000000          0.0          0.0
 9  Mater and the Ghostlight   2006-07-27   0.000000          0.0          0.0,
 "'Nombre del director': John Lasseter, 'retorno_total_director': 27.65437835777778")